# Synthea
Emergency Patients - First Encounter, 10k Patients

## Libraries and Constants

In [60]:
import subprocess

from pathlib import Path
import shutil
import pandas as pd

# Absolute paths to files and directories
BASE_DIRECTORY = (Path.cwd()).parent
SYNTHEA_PATH = Path(BASE_DIRECTORY / 'synthea-with-dependencies.jar')
RAW_DATA_PATH = Path(BASE_DIRECTORY / 'data/raw/analysis')
CSV_RAW_DATA_PATH = Path(RAW_DATA_PATH / 'csv')
PROCESSED_DATA_PATH = Path(BASE_DIRECTORY / 'data/processed/analysis')

# Shell command to run Synthea
RUN_SYNTHEA = f'java -jar {SYNTHEA_PATH}'

# CSV files that will be excluded from the generated data
EXCLUDED_CSV_FILES = [
    'careplans.csv',
    'claims.csv',
    'claims_transactions.csv',
    'organizations.csv',
    'patient_expenses.csv',
    'payer_transitions.csv',
    'payers.csv',
    'providers.csv',
    'supplies.csv'
]

# Number of patients to be generated
NUM_PATIENTS = 10_000

# If new Synthea data should be generated
GENERATE_DATA = False

# Data Generation

In [61]:
if(GENERATE_DATA):
    # Verify if the raw data directory exists
    if RAW_DATA_PATH.exists():
        # Delete the directory
        shutil.rmtree(RAW_DATA_PATH)

    # String containing the CSV files that will not be generated
    excluded_files = ','.join(EXCLUDED_CSV_FILES)

    # Define the shell command to generate the Synthea data
    command = RUN_SYNTHEA \
                + f' -p {NUM_PATIENTS}' \
                + f' --exporter.baseDirectory={RAW_DATA_PATH}' \
                + ' --exporter.csv.export=true' \
                + f' --exporter.csv.excluded_files={excluded_files}' \
                + ' --exporter.metadata.export=false' \
                + ' --exporter.fhir.export=false' \
                + ' --exporter.fhir.transaction_bundle=false' \
                + ' --exporter.hospital.fhir.export=false' \
                + ' --exporter.practitioner.fhir.export=false'

    # Run Synthea
    subprocess.run(command, shell=True)

## Data Processing

### Opening the Raw Data Files

In [62]:
# Get the raw data file names
raw_files = [file.name[:-4] for file in CSV_RAW_DATA_PATH.iterdir()]

# Load the DataFrames with raw data
dfs = dict()
for file in raw_files:
    dfs[file] = pd.read_csv(f'{CSV_RAW_DATA_PATH}/{file}.csv')

### Finding the Emergency Patients

In [63]:
# Drop unnecessary encounter columns
dfs['encounters'] = dfs['encounters'] \
    .drop(columns=[
        'ORGANIZATION', 'PROVIDER', 'PAYER', 'BASE_ENCOUNTER_COST',
        'TOTAL_CLAIM_COST', 'PAYER_COVERAGE'
    ])

# Filter only emergency encounters
dfs['encounters'] = dfs['encounters'] \
    .query('ENCOUNTERCLASS == "emergency"') \
    .query('REASONDESCRIPTION.notnull()') \
    .reset_index(drop=True)

# Get the identifiers (encounters and patients)
encounters_ids = dfs['encounters']['Id'].tolist()
patients_ids = dfs['encounters']['PATIENT'].tolist()

In [64]:
dfs['encounters']

,Id,START,STOP,PATIENT,ENCOUNTERCLASS,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,334a0b47-0430-24a9-09c4-8290d7d6fb98,2015-01-27T07:32:50Z,2015-01-27T08:32:50Z,c1ceda31-3ade-9760-437f-8eb454821bfc,emergency,50849002,Emergency room admission (procedure),1.256050e+08,Fracture of bone (disorder)
1,8a3e53d6-76e2-8315-a88f-cacfb649e1c4,2021-11-25T21:44:34Z,2021-11-25T22:44:34Z,e2335115-b5f4-8b93-81b1-cc51d3af1835,emergency,50849002,Emergency room admission (procedure),3.847090e+08,Sprain (morphologic abnormality)
2,8af14010-2158-c0b4-b83e-c61eafc083c0,2016-03-25T07:57:10Z,2016-03-25T08:57:10Z,c1ceda31-3ade-9760-437f-8eb454821bfc,emergency,50849002,Emergency room admission (procedure),3.847090e+08,Sprain (morphologic abnormality)
3,c5bf8b6f-53d2-aea3-7f69-bf81ff8693e5,2022-09-26T02:45:15Z,2022-09-26T03:45:15Z,bbd6deca-6ab7-9347-4a20-276331a8825f,emergency,50849002,Emergency room admission (procedure),3.126080e+08,Laceration - injury (disorder)
4,dfc0331b-7a1e-d635-5e16-13ec83b92d66,1982-01-14T13:51:52Z,1982-01-14T14:51:52Z,2f37267f-ef67-c7a5-aad5-0065e3eda89e,emergency,50849002,Emergency room admission (procedure),3.847090e+08,Sprain (morphologic abnormality)
...,...,...,...,...,...,...,...,...,...
19533,79f6e0e5-6c4f-7d64-783a-efcccb396ff8,2010-03-20T20:16:45Z,2010-03-20T21:16:45Z,09db5a5a-fb38-c484-14e1-7f35430691aa,emergency,50849002,Emergency room admission (procedure),1.256050e+08,Fracture of bone (disorder)
19534,679a0def-793a-f79e-2a2b-27c83283fc73,2013-05-27T21:00:34Z,2013-05-27T22:00:34Z,09db5a5a-fb38-c484-14e1-7f35430691aa,emergency,50849002,Emergency room admission (procedure),1.256050e+08,Fracture of bone (disorder)
19535,6904ea1f-43b3-a788-7375-4a20c98bac8f,1947-03-25T18:46:42Z,1947-03-25T21:26:16Z,291d0eaf-3b39-ce15-db1d-a2c5e29b8bb3,emergency,50849002,Emergency room admission (procedure),1.286130e+08,Seizure disorder
19536,ad730c7e-85f3-63e3-4fbf-9cda38fcb5c0,2013-09-30T18:46:42Z,2013-09-30T19:46:42Z,291d0eaf-3b39-ce15-db1d-a2c5e29b8bb3,emergency,182813001,Emergency treatment (procedure),1.241710e+14,Chronic intractable migraine without aura


### Filtering the Data Associated with the Infarction Emergency Encounters

In [65]:
# Filter data from DataFrames with encounter foreign key
for file in raw_files:
    if (file != 'encounters') and (file != 'patients'):
        dfs[file] = dfs[file] \
            .query('ENCOUNTER in @encounters_ids') \
            .reset_index(drop=True)

# # Drop unnecessary columns from immunizations, medications and procedures DataFrames
dfs['immunizations'] = dfs['immunizations'].drop(columns=['BASE_COST'])
dfs['medications'] = dfs['medications'] \
    .drop(columns=['BASE_COST', 'PAYER_COVERAGE', 'TOTALCOST'])
dfs['procedures'] = dfs['procedures'].drop(columns=['BASE_COST'])

# Filter data and drop unnecessary columns from the patients DataFrame
dfs['patients'] = dfs['patients'] \
    .drop(columns=['HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE', 'INCOME']) \
    .query('Id in @patients_ids') \
    .reset_index(drop=True)

### Writing CSV Files with the Processed Data

In [66]:
# Verify if the processed data directory exists
if PROCESSED_DATA_PATH.exists():
    # Delete the directory
    shutil.rmtree(PROCESSED_DATA_PATH)

# Create the processed data directory
PROCESSED_DATA_PATH.mkdir(parents=True)

# Write CSV files with processed data
for file in raw_files:
    dfs[file].to_csv(f'{PROCESSED_DATA_PATH}/{file}.csv', index=False)

## Data Analysis

### Treating the Encounters DataFrame

In [67]:
# Open the encounters data file
df_encounters = pd.read_csv(f'{PROCESSED_DATA_PATH}/encounters.csv')

# Drop unnecessary encounter columns
df_encounters_description = df_encounters \
    .drop(columns=[
        'START', 'STOP', 'PATIENT', 'ENCOUNTERCLASS', 'CODE', 'REASONCODE'
    ]) \
    .rename(columns={'Id': 'ENCOUNTER'})

# Print some of the encounters data
df_encounters_description.head()

,ENCOUNTER,DESCRIPTION,REASONDESCRIPTION
0,334a0b47-0430-24a9-09c4-8290d7d6fb98,Emergency room admission (procedure),Fracture of bone (disorder)
1,8a3e53d6-76e2-8315-a88f-cacfb649e1c4,Emergency room admission (procedure),Sprain (morphologic abnormality)
2,8af14010-2158-c0b4-b83e-c61eafc083c0,Emergency room admission (procedure),Sprain (morphologic abnormality)
3,c5bf8b6f-53d2-aea3-7f69-bf81ff8693e5,Emergency room admission (procedure),Laceration - injury (disorder)
4,dfc0331b-7a1e-d635-5e16-13ec83b92d66,Emergency room admission (procedure),Sprain (morphologic abnormality)


In [68]:
df_encounters \
    .groupby(by=['REASONDESCRIPTION'], as_index=False) \
    .agg(CASES=('REASONCODE', 'count')) \
    .sort_values(by=['CASES'], ascending=False) \
    .reset_index(drop=True)

,REASONDESCRIPTION,CASES
0,Normal pregnancy,2928
1,Drug overdose,1820
2,Fracture of bone (disorder),1729
3,Sprain (morphologic abnormality),1708
4,Laceration - injury (disorder),1593
5,Chronic intractable migraine without aura,1278
6,Chronic pain,1270
7,Impacted molars,1224
8,Concussion injury of brain (disorder),923
9,Myocardial infarction (disorder),553


### Verifying the Number of Conditions per Encounter

In [69]:
# Open the conditions data file
df_conditions = pd.read_csv(f'{PROCESSED_DATA_PATH}/conditions.csv')

# Print some of the conditions data
df_conditions.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2015-01-27,2015-02-28,c1ceda31-3ade-9760-437f-8eb454821bfc,334a0b47-0430-24a9-09c4-8290d7d6fb98,125605004,Fracture of bone (disorder)
1,2015-01-27,2015-02-28,c1ceda31-3ade-9760-437f-8eb454821bfc,334a0b47-0430-24a9-09c4-8290d7d6fb98,263102004,Fracture subluxation of wrist
2,2021-11-25,2021-12-10,e2335115-b5f4-8b93-81b1-cc51d3af1835,8a3e53d6-76e2-8315-a88f-cacfb649e1c4,384709000,Sprain (morphologic abnormality)
3,2021-11-25,2021-12-10,e2335115-b5f4-8b93-81b1-cc51d3af1835,8a3e53d6-76e2-8315-a88f-cacfb649e1c4,44465007,Sprain of ankle
4,2016-03-25,2016-04-15,c1ceda31-3ade-9760-437f-8eb454821bfc,8af14010-2158-c0b4-b83e-c61eafc083c0,384709000,Sprain (morphologic abnormality)


In [70]:
# Aggregate data using encounter codes
df_conditions_agg = df_conditions \
    .groupby(by=['ENCOUNTER'], as_index=False) \
    .agg(
        NUM_CONDITIONS=('CODE', pd.Series.nunique),
        CONDITIONS=('DESCRIPTION', 'unique')
    ) \
    .sort_values(by=['NUM_CONDITIONS'], ascending=False) \
    .reset_index(drop=True)

# Save a CSV file with the aggregation result
df_conditions_agg.to_csv(f'{PROCESSED_DATA_PATH}/agg_conditions.csv', index=False)

# Print some of the aggregation result
df_conditions_agg

,ENCOUNTER,NUM_CONDITIONS,CONDITIONS
0,f081321a-abc9-aa34-ad34-0f03f69abb60,5,"[Fracture of bone (disorder), Fracture subluxa..."
1,bd4faa77-ae20-88e5-9625-002522c492a4,4,"[Fracture of bone (disorder), Fracture of clav..."
2,65d5a6eb-bab9-04e3-46a8-634ea78393d6,4,"[Fracture of bone (disorder), Closed fracture ..."
3,9607d609-c04e-9ab9-f61b-6a971e89d13f,4,"[Fracture of bone (disorder), Fracture subluxa..."
4,3683800e-c81c-83b9-25c0-4536da5636bf,3,"[Sprain (morphologic abnormality), Sprain of a..."
...,...,...,...
12285,3b058ab2-7c49-5cec-b5c5-45377736e7f0,1,[Drug overdose]
12286,3b074f13-83ca-8929-b875-3ed12228adb9,1,[Drug overdose]
12287,a8b42821-919b-ce41-73e4-e29284cb1885,1,[Sepsis (disorder)]
12288,a8b4214e-9b3a-4b5e-a214-8ea3c12bc77b,1,[Chronic obstructive bronchitis (disorder)]


### Verifying the Number of Observations per Encounter

In [71]:
# Open the observations data file
df_observations = pd.read_csv(f'{PROCESSED_DATA_PATH}/observations.csv')

# Print some of the observations data
df_observations.head()

,DATE,PATIENT,ENCOUNTER,CATEGORY,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,1996-08-01T18:37:09Z,1cd30fd1-b956-bee3-b7ba-9c0ac80f5588,793a6f40-e91f-b008-a287-88d6b7ee0a8c,vital-signs,2708-6,Oxygen saturation in Arterial blood,84.0,%,numeric
1,1996-08-01T18:37:09Z,1cd30fd1-b956-bee3-b7ba-9c0ac80f5588,793a6f40-e91f-b008-a287-88d6b7ee0a8c,vital-signs,29463-7,Body Weight,84.3,kg,numeric
2,1996-08-01T18:37:09Z,1cd30fd1-b956-bee3-b7ba-9c0ac80f5588,793a6f40-e91f-b008-a287-88d6b7ee0a8c,laboratory,33762-6,NT-proBNP,996.5,pg/mL,numeric
3,1996-08-01T18:37:09Z,1cd30fd1-b956-bee3-b7ba-9c0ac80f5588,793a6f40-e91f-b008-a287-88d6b7ee0a8c,laboratory,6690-2,Leukocytes [#/volume] in Blood by Automated count,4.9,10*3/uL,numeric
4,1996-08-01T18:37:09Z,1cd30fd1-b956-bee3-b7ba-9c0ac80f5588,793a6f40-e91f-b008-a287-88d6b7ee0a8c,laboratory,789-8,Erythrocytes [#/volume] in Blood by Automated ...,4.1,10*6/uL,numeric


In [72]:
# Aggregate data using encounter codes
df_observations_agg = df_observations \
    .groupby(by=['ENCOUNTER'], as_index=False) \
    .agg(
        NUM_CATEGORIES=('CATEGORY', pd.Series.nunique),
        CATEGORIES=('CATEGORY', 'unique'),
        NUM_OBSERVATIONS=('CODE', pd.Series.nunique),
        OBSERVATIONS=('DESCRIPTION', 'unique')
    ) \
    .sort_values(by=['NUM_CATEGORIES', 'NUM_OBSERVATIONS'], ascending=False) \
    .reset_index(drop=True)

# Save a CSV file with the aggregation result
df_observations_agg.to_csv(f'{PROCESSED_DATA_PATH}/agg_observations.csv', index=False)

# Print some of the aggregation result
df_observations_agg

,ENCOUNTER,NUM_CATEGORIES,CATEGORIES,NUM_OBSERVATIONS,OBSERVATIONS
0,4cb64798-606e-7170-33b5-6f7a35915312,3,"[exam, vital-signs, laboratory]",49,"[Emergency severity index, Diastolic Blood Pre..."
1,03309711-368c-ae72-4d1e-59439e823f9e,3,"[exam, vital-signs, laboratory]",48,"[Emergency severity index, Diastolic Blood Pre..."
2,1ac2d4d7-6370-ec59-083c-6641418c5afe,3,"[exam, vital-signs, laboratory]",48,"[Emergency severity index, Diastolic Blood Pre..."
3,344bdd5c-a3d2-a869-f55d-ecf10757f0a5,3,"[exam, vital-signs, laboratory]",48,"[Emergency severity index, Diastolic Blood Pre..."
4,3d03b169-5ac7-4342-7e5e-f82bbb92f358,3,"[exam, vital-signs, laboratory]",48,"[Emergency severity index, Diastolic Blood Pre..."
...,...,...,...,...,...
4362,ff6e082b-ca6a-754c-66c3-257c6af740a2,1,[vital-signs],1,[Pain severity - 0-10 verbal numeric rating [S...
4363,ff7f4758-57a6-0121-8f70-9e08b8c971ef,1,[vital-signs],1,[Pain severity - 0-10 verbal numeric rating [S...
4364,ff8fd48b-352d-fbde-1e96-f857b5df34e9,1,[vital-signs],1,[Pain severity - 0-10 verbal numeric rating [S...
4365,ffa0401f-4050-2d75-ad6e-2963b26bd68c,1,[vital-signs],1,[Pain severity - 0-10 verbal numeric rating [S...


### Verifying the Number of Conditions and Observations per Encounter

In [73]:
# Merge aggregation result for conditions and observations
df_merged_agg = df_conditions_agg \
    .merge(
        right=df_observations_agg,
        how='left',
        on='ENCOUNTER'
    ) \
    .sort_values(
        by=['NUM_CONDITIONS', 'NUM_CATEGORIES', 'NUM_OBSERVATIONS'],
        ascending=False
    ) \
    .fillna({
        'NUM_CATEGORIES': 0,
        'CATEGORIES': '',
        'NUM_OBSERVATIONS': 0,
        'OBSERVATIONS': ''
    }) \
    .astype({'NUM_CATEGORIES': int, 'NUM_OBSERVATIONS': int})

# Save a CSV file with the merge result
df_merged_agg.to_csv(f'{PROCESSED_DATA_PATH}/agg_conditions_observations.csv', index=False)

# Print some of the merge result
df_merged_agg

,ENCOUNTER,NUM_CONDITIONS,CONDITIONS,NUM_CATEGORIES,CATEGORIES,NUM_OBSERVATIONS,OBSERVATIONS
0,f081321a-abc9-aa34-ad34-0f03f69abb60,5,"[Fracture of bone (disorder), Fracture subluxa...",2,"[procedure, survey]",26,"[DXA Radius and Ulna [T-score] Bone density, W..."
1,bd4faa77-ae20-88e5-9625-002522c492a4,4,"[Fracture of bone (disorder), Fracture of clav...",1,[procedure],1,[DXA Radius and Ulna [T-score] Bone density]
2,65d5a6eb-bab9-04e3-46a8-634ea78393d6,4,"[Fracture of bone (disorder), Closed fracture ...",1,[procedure],1,[DXA Radius and Ulna [T-score] Bone density]
3,9607d609-c04e-9ab9-f61b-6a971e89d13f,4,"[Fracture of bone (disorder), Fracture subluxa...",1,[procedure],1,[DXA Radius and Ulna [T-score] Bone density]
24,da555dee-4880-64e4-e2fb-e35ac6c4ef2c,3,"[Sepsis (disorder), Septic shock (disorder), A...",3,"[laboratory, exam, vital-signs]",7,[Gram positive blood culture panel by Probe in...
...,...,...,...,...,...,...,...
12285,3b058ab2-7c49-5cec-b5c5-45377736e7f0,1,[Drug overdose],0,,0,
12286,3b074f13-83ca-8929-b875-3ed12228adb9,1,[Drug overdose],0,,0,
12287,a8b42821-919b-ce41-73e4-e29284cb1885,1,[Sepsis (disorder)],0,,0,
12288,a8b4214e-9b3a-4b5e-a214-8ea3c12bc77b,1,[Chronic obstructive bronchitis (disorder)],0,,0,


In [74]:
# Merge aggregation result for conditions and observations
df_merged_aggs = df_encounters_description \
    .merge(
        right=df_merged_agg,
        how='left',
        on='ENCOUNTER'
    ) \
    .sort_values(
        by=['NUM_CONDITIONS', 'NUM_CATEGORIES', 'NUM_OBSERVATIONS'],
        ascending=False
    ) \
    .fillna({
        'NUM_CONDITIONS': 0,
        'CONDITIONS': '',
        'NUM_CATEGORIES': 0,
        'CATEGORIES': '',
        'NUM_OBSERVATIONS': 0,
        'OBSERVATIONS': ''
    }) \
    .astype({'NUM_CATEGORIES': int, 'NUM_OBSERVATIONS': int})

# Save a CSV file with the merge result
# df_merged_aggs.to_csv(f'{PROCESSED_DATA_PATH}/agg_conditions_observations.csv', index=False)

# Print some of the merge result
df_merged_aggs

,ENCOUNTER,DESCRIPTION,REASONDESCRIPTION,NUM_CONDITIONS,CONDITIONS,NUM_CATEGORIES,CATEGORIES,NUM_OBSERVATIONS,OBSERVATIONS
15136,f081321a-abc9-aa34-ad34-0f03f69abb60,Emergency room admission (procedure),Fracture of bone (disorder),5.0,"[Fracture of bone (disorder), Fracture subluxa...",2,"[procedure, survey]",26,"[DXA Radius and Ulna [T-score] Bone density, W..."
1035,bd4faa77-ae20-88e5-9625-002522c492a4,Emergency room admission (procedure),Fracture of bone (disorder),4.0,"[Fracture of bone (disorder), Fracture of clav...",1,[procedure],1,[DXA Radius and Ulna [T-score] Bone density]
9358,65d5a6eb-bab9-04e3-46a8-634ea78393d6,Emergency room admission (procedure),Fracture of bone (disorder),4.0,"[Fracture of bone (disorder), Closed fracture ...",1,[procedure],1,[DXA Radius and Ulna [T-score] Bone density]
14685,9607d609-c04e-9ab9-f61b-6a971e89d13f,Emergency room admission (procedure),Fracture of bone (disorder),4.0,"[Fracture of bone (disorder), Fracture subluxa...",1,[procedure],1,[DXA Radius and Ulna [T-score] Bone density]
245,d043cdd4-7fd6-466f-0cf2-f281f0549bbb,Encounter for problem,Sepsis (disorder),3.0,"[Sepsis (disorder), Septic shock (disorder), A...",3,"[laboratory, exam, vital-signs]",7,[Gram positive blood culture panel by Probe in...
...,...,...,...,...,...,...,...,...,...
19517,1b675ad0-0ffa-2203-524d-8a4cf0f10d5b,Emergency treatment (procedure),Impacted molars,0.0,,0,,0,
19518,5e4a7623-c97a-caa0-72a1-d81307dda1d7,Emergency hospital admission (procedure),Chronic pain,0.0,,0,,0,
19519,adb23b8e-9a61-5e5d-a70e-d9a5886fc12a,Emergency treatment (procedure),Impacted molars,0.0,,0,,0,
19520,5f8aacb6-0c0e-b82f-6e66-8ecaa841d239,Emergency treatment (procedure),Chronic intractable migraine without aura,0.0,,0,,0,


In [75]:
df_merged_aggs \
    .groupby(by=['REASONDESCRIPTION'], as_index=False) \
    .agg(
        CASES=('ENCOUNTER', 'count'),
        MEAN_CONDITIONS=('NUM_CONDITIONS', 'mean'),
        STD_CONDITIONS=('NUM_CONDITIONS', 'std'),
        MEAN_OBSERVATIONS=('NUM_OBSERVATIONS', 'mean'),
        STD_OBSERVATIONS=('NUM_OBSERVATIONS', 'std')
    ) \
    .sort_values(['MEAN_CONDITIONS', 'MEAN_OBSERVATIONS'], ascending=[False, True]) \
    .reset_index(drop=True)

,REASONDESCRIPTION,CASES,MEAN_CONDITIONS,STD_CONDITIONS,MEAN_OBSERVATIONS,STD_OBSERVATIONS
0,Fracture of bone (disorder),1729,2.035859,0.529116,0.340659,0.777412
1,Sprain (morphologic abnormality),1708,2.008197,0.123143,0.043911,0.995510
2,Laceration - injury (disorder),1593,2.003766,0.146091,0.030760,0.835568
3,Injury of knee (disorder),277,2.003610,0.104194,0.007220,0.120168
4,Gunshot wound (disorder),62,2.000000,0.000000,0.000000,0.000000
5,Injury of neck (disorder),531,2.000000,0.000000,0.000000,0.000000
6,Seizure disorder,525,1.948571,0.984238,0.000000,0.000000
7,Acute Cholecystitis,31,1.935484,0.359211,23.225806,4.310527
8,Concussion injury of brain (disorder),923,1.894908,0.310353,0.024919,0.757054
9,Burn injury (morphologic abnormality),503,1.876740,0.468867,0.000000,0.000000
